In [1]:
# import dependencies
import win32com.client as wc
import xml.etree.ElementTree as ET
import pandas as pd
import mysql.connector as mc
import datetime as dt
from datetime import datetime
import pyodbc
from pdfrw import PdfReader, PdfWriter
import pdfrw 
import os
from pdf2image import convert_from_path
import img2pdf
from PIL import Image

# Get sql user and password
from configTest import mysql_host, mysql_u, mysql_pw
from configTest import vgc_host, vgc_u, vgc_pw
from configTest import svr, db, sql_u, sql_pw

In [2]:
# create batch_id
now = datetime.now()
batch_id = now.strftime("%Y%m%d%H%M%S")

In [3]:
# PDF Concatenation Function
def ConCat_pdf (file_list, outfn):
    writer = PdfWriter()
    for inputfn in file_list:
        writer.addpages(PdfReader(inputfn).pages)

    writer.write(outfn)
    return outfn

In [4]:
# Delete Function
def delete_file(del_file_path):
    if os.path.exists(del_file_path):
        os.remove(del_file_path)
    else: print (f"{del_file_path} does not exist")

In [5]:
# Create PDF Function
def fill_pdf(input_pdf_path, output_pdf_path, data_dict):
    ANNOT_KEY = '/Annots'
    ANNOT_FIELD_KEY = '/T'
    ANNOT_VAL_KEY = '/V'
    ANNOT_RECT_KEY = '/Rect'
    SUBTYPE_KEY = '/Subtype'
    WIDGET_SUBTYPE_KEY = '/Widget'

    template_pdf = pdfrw.PdfReader(input_pdf_path)
    
    for page in template_pdf.pages:
        annotations = page[ANNOT_KEY]
        for annotation in annotations:
            if annotation[SUBTYPE_KEY] == WIDGET_SUBTYPE_KEY:
                if annotation[ANNOT_FIELD_KEY]:
                    key = annotation[ANNOT_FIELD_KEY][1:-1]
                    if key in data_dict.keys():
                        if type(data_dict[key]) == bool:
                            if data_dict[key] == True:
                                annotation.update(pdfrw.PdfDict(
                                    AS=pdfrw.PdfName('Yes')))
                        else:
                            annotation.update(
                                pdfrw.PdfDict(V='{}'.format(data_dict[key]))
                            )
                            annotation.update(pdfrw.PdfDict(AP=''))
    template_pdf.Root.AcroForm.update(pdfrw.PdfDict(NeedAppearances=pdfrw.PdfObject('true')))

    pdfrw.PdfWriter().write(output_pdf_path, template_pdf)

In [6]:
# Flatten PDF function
def flatten_pdf(flat_output, img_file, file_list):
    # Fillable PDF to Image
    images = convert_from_path(flat_output, dpi=300, size=(2550,3300))
    for i in range(len(images)):
   
    # Save pages as images in the pdf
        images[i].save(img_file + '.png', 'PNG')
    
    # Delete Fillable PDF
    delete_file(img_file + '.pdf')
    
    # opening image
    image_file = Image.open(img_file + '.png')
    
    # Image to Flat PDF
    # define paper size
    letter = (img2pdf.in_to_pt(8.5), img2pdf.in_to_pt(11))
    layout = img2pdf.get_layout_fun(letter)
    # converting into chunks using img2pdf
    pdf_bytes = img2pdf.convert(image_file.filename, layout_fun=layout)
    
    # opening or creating pdf file
    flat_pdf = f"{img_file}.pdf"
    file = open(flat_pdf, "wb")
    
    # writing pdf files with chunks
    file.write(pdf_bytes)
    
    # Add file name to file_name list
    file_list.append(flat_pdf)

    # closing image file
    image_file.close()
    
     # Delete Fillable PDF
    delete_file(img_file + '.png')

    # closing pdf file
    file.close()

    return file_list

In [7]:
# GAP Letter function
def gap_letter(template_df):

    gap_path = 'letters/staging/gap/'
    pdf_template = "letters/pdf_templates/GAP_letter_template.pdf"
    # template_pdf = pdfrw.PdfReader(pdf_template)

    template_df['payment_amount'] = template_df['payment_amount'].map('${:,.2f}'.format)
    template_df['loss_date'] = pd.to_datetime(template_df['loss_date']).dt.strftime('%B %d, %Y')
    template_df['StateDesc'] = template_df['StateDesc'].astype(str).replace({'None':''})
    template_df['StateCode'] = template_df['StateCode'].astype(str).replace({'None':''})
    template_df['f_lang'] = template_df['f_lang'].astype(str).replace({'None':''})
    # letter_date = f"{datetime.date.today():%B %d, %Y}"
    letter_date = f"{datetime.now():%B %d, %Y}"   

    # now = datetime.now()
    # batch_id = now.strftime("%Y%m%d%H%M%S") 

    file_list = []

    for index, row in template_df.iterrows():

        # empty dict
        data_dict = {}
        # store field data in dictionary
        data_dict = {
            'Date': letter_date,
            'Lender': template_df.loc[index]['alt_name'],
            'Contact': template_df.loc[index]['contact'],
            'Address': template_df.loc[index]['address1'],
            'City_St_Zip': f"{template_df.loc[index]['city']}, {template_df.loc[index]['state']} {template_df.loc[index]['zip']}",
            'Lender2': template_df.loc[index]['alt_name'],
            'Borrower': f"{template_df.loc[index]['first']} {template_df.loc[index]['last']}",
            'Claim_Nbr': template_df.loc[index]['claim_nbr'],
            'Acct_Nbr': template_df.loc[index]['acct_number'],
            'DOL': template_df.loc[index]['loss_date'],
            'GAP_Amt': template_df.loc[index]['payment_amount'],
            'State': template_df.loc[index]['StateDesc'],
            'St_Code': template_df.loc[index]['StateCode'],
            'Fraud': template_df.loc[index]['f_lang'],
        }

        # store paths as variables
        output_file = f"{template_df.loc[index]['claim_nbr']}.pdf"
        output_path_fn = f"{gap_path}{output_file}"

        fill_pdf(pdf_template, output_path_fn, data_dict)

        # Set File Paths
        flat_output = f"{os.path.dirname(os.path.abspath(output_file))}\{gap_path}\{output_file}"
        img_file = f"{os.path.dirname(os.path.abspath(output_file))}\{gap_path}\{template_df.loc[index]['claim_nbr']}"

        # Flatten pdf using flatten_pdf function
        file_list = flatten_pdf(flat_output, img_file, file_list) 

    return file_list


In [8]:
# GAP Calculation function


In [9]:
# GAP Plus function


In [10]:
# TotalRestart Letter function


In [11]:
# TotalRestart Calculation function


In [12]:
# connect to DB
cnx = mc.connect(user=vgc_u, password=vgc_pw,
                host=vgc_host,
                database='visualgap_claims')
cursor = cnx.cursor()

In [13]:
# sql query for GAP claims that are RTBP
sql_file = '''
            SELECT c.claim_id, c.claim_nbr, c.carrier_id, cl.alt_name, cl.dealer_securityId, cl.contact, cl.address1,
                cl.city, cl.state, cl.zip, cl.payment_method, cb.first, cb.last, 
                IF(sq.gap_amt_paid > 0, 2,1) AS pymt_type_id, 
                IF(sq.gap_amt_paid > 0, ROUND(cc.gap_payable - sq.gap_amt_paid,2), cc.gap_payable) AS gap_due,
                cb.acct_number
            FROM claims c
            INNER JOIN claim_lender cl
                USING (claim_id)
            INNER JOIN claim_borrower cb
                USING (claim_id)
            INNER JOIN claim_calculations cc
                USING (claim_id)
            INNER JOIN claim_status cs
                ON (c.status_id = cs.status_id)
            LEFT JOIN (SELECT cp.claim_id, SUM(cp.payment_amount) AS gap_amt_paid
                    FROM claim_payments cp
                    INNER JOIN (SELECT c.claim_id
                                FROM claims c
                                INNER JOIN claim_status cs
                                    ON (c.status_id = cs.status_id)
                                WHERE cs.status_desc_id = 8) rtbp_sq
                        USING (claim_id)
                    WHERE payment_category_id = 1
                    GROUP BY cp.claim_id) sq
                ON (c.claim_id = sq.claim_id)
            WHERE cs.status_desc_id = 8;
            '''

In [14]:
# execute sql
cursor.execute(sql_file)

In [15]:
# save query results as DF
df = pd.DataFrame(cursor.fetchall())

In [16]:
# add column names
df_cols = ['claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'dealer_securityId', 'contact', 'address1', 'city', 'state', 'zip', 
                            'pymt_method', 'first', 'last', 'pymt_type_id', 'amount', 'acct_number']

df.columns = df_cols

In [17]:
# sql query for PLUS claims that are RTBP
sql_file_plus = '''
                SELECT sqp.claim_id, c.claim_nbr, c.carrier_id, cl.alt_name, cl.dealer_securityId, cl.contact, 
                    cl.address1, cl.city, cl.state, cl.zip, cl.payment_method, cb.first, cb.last, 
                    1 AS pymt_type_id, 
                    IF(cl.customer_securityId = 9401, 1500, 1000) AS gap_plus_due, cb.acct_number
                FROM claims c
                INNER JOIN claim_lender cl
                    USING (claim_id)
                INNER JOIN claim_borrower cb
                    USING (claim_id)
                INNER JOIN (SELECT pb.claim_id
                            FROM claim_plus_benefit pb
                            WHERE status_desc_id = 8) sqp
                    USING (claim_id)
                INNER JOIN (SELECT c.claim_id
                            FROM claims c
                            INNER JOIN claim_status cs
                                ON (c.status_id = cs.status_id)  
                            WHERE cs.status_desc_id = 8
                                OR cs.status_desc_id = 4) sqg
                    USING (claim_id);
            '''

In [18]:
# execute sql
cursor.execute(sql_file_plus)

In [19]:
# save query results as DF
df2 = pd.DataFrame(cursor.fetchall())

In [20]:
# add column names
df2_cols = ['claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'dealer_securityId', 'contact', 'address1', 'city', 'state', 'zip', 
                            'pymt_method', 'first', 'last', 'pymt_type_id', 'amount', 'acct_number']

df2.columns = df2_cols

In [21]:
# sql query for TotalRestart claims that are RTBP
sql_file_tr = '''
            SELECT sqp.claim_id, c.claim_nbr, c.carrier_id, cl.alt_name, cl.dealer_securityId, cl.contact, cl.address1, 
            cl.city, cl.state, cl.zip, 'Check' AS payment_method, cb.first, cb.last, 1 AS pymt_type_id, 
            ctr.totalrestart_payable AS tr_due, cb.acct_number
            FROM claims c
            INNER JOIN claim_lender cl
                USING (claim_id)
            INNER JOIN claim_borrower cb
                USING (claim_id)
            INNER JOIN (SELECT pb.claim_id
                        FROM claim_totalrestart pb
                        WHERE status_desc_id = 8) sqp
                USING (claim_id)
            INNER JOIN claim_totalrestart ctr
                USING (claim_id)
             '''

In [22]:
# execute sql
cursor.execute(sql_file_tr)

In [23]:
# save query results as DF
df3 = pd.DataFrame(cursor.fetchall())

In [24]:
# add column names
df3_cols = ['claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'dealer_securityId', 'contact', 'address1', 'city', 'state', 'zip', 
                            'pymt_method', 'first', 'last', 'pymt_type_id', 'amount', 'acct_number']

df3.columns = df3_cols

In [25]:
# close mysql connection
cursor.close()
cnx.close()

In [26]:
# get expense account name
cnx = mc.connect(user=mysql_u, password=mysql_pw,
                host=mysql_host,
                database='claim_qb_payments')
cursor = cnx.cursor()

In [27]:
# GAP
# convert columns list to string
cols = ", ".join(df_cols)

# insert DF into the ready_to_be_paid table of the claim_qb_payments database
for x,rows in df.iterrows():

    sql_file2 = '''INSERT INTO ready_to_be_paid ({columns}, payment_category_id, check_nbr, batch_id, qb_txnid, toVGC) VALUES ({claim_id}, "{claim_nbr}", {carrier_id},"{lender_name}", "{lender_id}","{contact}", 
                "{address1}", "{city}", "{state}", "{zip}", "{pymt_method}", "{first}", "{last}", {pymt_type_id}, {amount}, "{acct_nbr}", 1, 0, {batchId}, 0, 0);'''.format(columns=cols, 
                claim_id=rows['claim_id'], claim_nbr=rows['claim_nbr'], carrier_id=rows['carrier_id'], lender_name=rows['lender_name'], lender_id=rows['dealer_securityId'], 
                contact=rows['contact'], address1=rows['address1'], city=rows['city'], state=rows['state'], zip=rows['zip'], 
                pymt_method=rows['pymt_method'], first = rows['first'], last = rows['last'], pymt_type_id = rows['pymt_type_id'], 
                amount = rows['amount'], acct_nbr = rows['acct_number'], batchId = batch_id)
     
    # execute and commit sql
    cursor.execute(sql_file2)
    cnx.commit()

In [28]:
# PLUS
# convert columns list to string
cols = ", ".join(df2_cols)

# insert DF into the ready_to_be_paid table of the claim_qb_payments database
for x,rows in df2.iterrows():
    sql_file2_plus = '''INSERT INTO ready_to_be_paid ({columns}, payment_category_id, check_nbr, batch_id, qb_txnid, toVGC) VALUES ({claim_id}, "{claim_nbr}", {carrier_id},"{lender_name}", "{lender_id}","{contact}", 
                "{address1}", "{city}", "{state}", "{zip}", "{pymt_method}", "{first}", "{last}", {pymt_type_id}, {amount}, "{acct_nbr}", 2, 0, {batchId}, 0, 0);'''.format(columns=cols, 
                claim_id=rows['claim_id'], claim_nbr=rows['claim_nbr'], carrier_id=rows['carrier_id'], lender_name=rows['lender_name'], lender_id=rows['dealer_securityId'], 
                contact=rows['contact'], address1=rows['address1'], city=rows['city'], state=rows['state'], zip=rows['zip'], 
                pymt_method=rows['pymt_method'], first = rows['first'], last = rows['last'], pymt_type_id = rows['pymt_type_id'], 
                amount = rows['amount'], acct_nbr = rows['acct_number'], batchId = batch_id)
          
    # execute and commit sql
    cursor.execute(sql_file2_plus)
    cnx.commit()

In [29]:
# TOTALRESTART
# convert columns list to string
cols = ", ".join(df3_cols)

# insert DF into the ready_to_be_paid table of the claim_qb_payments database
for x,rows in df3.iterrows():
    sql_file2_tr = '''INSERT INTO ready_to_be_paid ({columns}, payment_category_id, check_nbr, batch_id, qb_txnid, toVGC) VALUES ({claim_id}, "{claim_nbr}", {carrier_id},"{lender_name}", "{lender_id}","{contact}", 
                "{address1}", "{city}", "{state}", "{zip}", "{pymt_method}", "{first}", "{last}", {pymt_type_id}, {amount}, "{acct_nbr}", 3, 0, {batchId}, 0, 0);'''.format(columns=cols, 
                claim_id=rows['claim_id'], claim_nbr=rows['claim_nbr'], carrier_id=rows['carrier_id'], lender_name=rows['lender_name'], lender_id=rows['dealer_securityId'], 
                contact=rows['contact'], address1=rows['address1'], city=rows['city'], state=rows['state'], zip=rows['zip'], 
                pymt_method=rows['pymt_method'], first = rows['first'], last = rows['last'], pymt_type_id = rows['pymt_type_id'], 
                amount = rows['amount'], acct_nbr = rows['acct_number'], batchId = batch_id)
          
    # execute and commit sql
    cursor.execute(sql_file2_tr)
    cnx.commit()

In [30]:
# create select query to pull current batch with ID
sql_file3 = '''SELECT rtbp_id, claim_id, claim_nbr, carrier_id, lender_name, dealer_securityId, contact, address1, city, 
                     state, zip, pymt_method, first, last, pymt_type_id, amount, payment_category_id, check_nbr, batch_id, 
                     qb_txnid, acct_number
              FROM ready_to_be_paid
              WHERE batch_id = {batchId}
              
              ORDER BY payment_category_id, claim_id;'''.format(batchId = batch_id)

In [31]:
# execute sql
cursor.execute(sql_file3)

In [32]:
# save query results as DF
pymts_df = pd.DataFrame(cursor.fetchall())

# add column names
pymts_df_cols = ['rtbp_id', 'claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'dealer_securityId', 'contact', 'address1', 'city', 'state', 'zip', 
                    'pymt_method', 'first', 'last', 'pymt_type_id', 'amount','payment_category_id', 'check_nbr', 'batch_id', 'qb_txnid', 
                    'acct_number']

pymts_df.columns = pymts_df_cols

In [33]:
pymts_df

,rtbp_id,claim_id,claim_nbr,carrier_id,lender_name,dealer_securityId,contact,address1,city,state,...,pymt_method,first,last,pymt_type_id,amount,payment_category_id,check_nbr,batch_id,qb_txnid,acct_number
0,199,1466,202109172488,9,Charlotte Metro Credit Union,22260,CMCU Contact,8829 Chapel Square Dr (Jareds Office),Cincinnati,OH,...,Check,Sam,Hunt,1,5302.64,1,0,20211124143822,0,
1,202,1469,202110182491,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,...,Check,TIFFANY,LEGO,1,2144.05,1,0,20211124143822,0,
2,205,1470,202110182492,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,...,Check,MARY,SCHEIBLE,1,0.00,1,0,20211124143822,0,6343056-50
3,200,1471,202110182493,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,...,ACH-GL,ANDREW,FLYNN,1,0.00,1,0,20211124143822,0,
4,201,1472,202110182494,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,...,ACH-GL,TYSHAUN,GRANT,1,5138.04,1,0,20211124143822,0,250510182
5,203,1473,202110182495,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,...,Check,ALEX,MITCHELL,1,0.00,1,0,20211124143822,0,
6,209,1474,202110192496,9,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,...,Check,JOSHUA,WELCH,1,0.00,1,0,20211124143822,0,
7,204,1475,202110192497,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,...,ACH-GL,OCTAVIO,PARTIDA,1,653.87,1,0,20211124143822,0,187895-01
8,206,1476,202110192498,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,...,ACH-GL,SHERRY,VOGT,1,0.00,1,0,20211124143822,0,136324-02L
9,210,1477,202110192499,9,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,...,Check,BRIAN,CHANDLER,2,191.17,1,0,20211124143822,0,250505961


In [34]:
# sql query for expense accounts in DB
sql_file4 = '''
    SELECT carrier_id, qb_fullname
    FROM qb_accounts
    WHERE account_type = 'Expense';
    '''

In [35]:
# execute sql
cursor.execute(sql_file4)
# save query results as DF
expense_df = pd.DataFrame(cursor.fetchall())
# add column names to DF
col_names = ['carrier_id', 'expense']
expense_df.columns = col_names

In [36]:
# sql query for checking accounts in DB
sql_file5 = '''
    SELECT carrier_id, qb_fullname
    FROM qb_accounts
    WHERE account_type = 'Checking';
    '''

In [37]:
# execute sql
cursor.execute(sql_file5)
# save query results as DF
checking_df = pd.DataFrame(cursor.fetchall())
# add column names to DF
col_names = ['carrier_id', 'checking']
checking_df.columns = col_names

In [38]:
# Merge expense account name into df
pymts_df = pymts_df.merge(expense_df, left_on='carrier_id', right_on='carrier_id').copy()

In [39]:
# Merge checking account name into df
pymts_df = pymts_df.merge(checking_df, left_on='carrier_id', right_on='carrier_id').copy()

In [40]:
# Create sql server connection
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+svr+';DATABASE='+db+';UID='+sql_u+';PWD='+ sql_pw)

In [41]:
# create query
sql_svr_file = '''SELECT VGSecurityId, QB_ListID
                  FROM business_entity
                  WHERE QB_ListID IS NOT NULL;
               '''
# execute query
listid_df = pd.read_sql(sql_svr_file, cnxn) 

In [42]:
# TEMPORARY ########################################################################################
# Convert test dealer_securityId to Production dealer_securityId
pymts_df['dealer_securityId'].replace({22260:46724,21945:52715}, inplace=True)
# TEMPORARY ########################################################################################

In [43]:
# TOTALRESTART #####################################################################################
# Update TR claims with the correct 'checking' & 'expense' names and possibly QB_ListID
# maybe change carrier to pull correct 'checking' & 'expense' names and possibly QB_ListID??
####################################################################################################

In [44]:
# Merge QB_ListID into df
pymts_df = pymts_df.merge(listid_df, left_on='dealer_securityId', right_on='VGSecurityId').copy()

In [45]:
# TEMPORARY ########################################################################################
# temp Check order (claim_id, payment_category_id)
pymts_df
# TEMPORARY ########################################################################################

,rtbp_id,claim_id,claim_nbr,carrier_id,lender_name,dealer_securityId,contact,address1,city,state,...,amount,payment_category_id,check_nbr,batch_id,qb_txnid,acct_number,expense,checking,VGSecurityId,QB_ListID
0,199,1466,202109172488,9,Charlotte Metro Credit Union,46724,CMCU Contact,8829 Chapel Square Dr (Jareds Office),Cincinnati,OH,...,5302.64,1,0,20211124143822,0,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724
1,200,1471,202110182493,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,...,0.00,1,0,20211124143822,0,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724
2,201,1472,202110182494,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,...,5138.04,1,0,20211124143822,0,250510182,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724
3,204,1475,202110192497,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,...,653.87,1,0,20211124143822,0,187895-01,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724
4,206,1476,202110192498,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,...,0.00,1,0,20211124143822,0,136324-02L,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724
5,212,1490,202111092512,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,...,0.00,1,0,20211124143822,0,123-456,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724
6,214,1475,202110192497,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,...,1000.00,2,0,20211124143822,0,187895-01,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724
7,215,1490,202111092512,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,...,4000.00,3,0,20211124143822,0,123-456,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724
8,209,1474,202110192496,9,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,...,0.00,1,0,20211124143822,0,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),52715,80000002-1628088752
9,210,1477,202110192499,9,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,...,191.17,1,0,20211124143822,0,250505961,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),52715,80000002-1628088752


In [ ]:
# payments greater than 0 df
qb_pymts_df = pymts_df.loc[pymts_df['amount'] > 0]
qb_pymts_df

In [ ]:
# Connect to Quickbooks
sessionManager = wc.Dispatch("QBXMLRP2.RequestProcessor")    
sessionManager.OpenConnection('', 'Test qbXML Request')
ticket = sessionManager.BeginSession("", 2)

In [ ]:
# create qbxml query to add payments to QB
pay_date = f"{dt.date.today():%Y-%m-%d}"

for index, row in pymts_df.iterrows():
    if row['payment_category_id'] == 1:
        if row['pymt_type_id'] == 2:
            pymt_type = 'Additional GAP Claim Pymt'
        else: 
            pymt_type = 'GAP Claim'

    elif row['payment_category_id'] == 2:
        if row['pymt_type_id'] == 2:
            pymt_type = 'Additional GAP Plus Pymt'
        else: 
            pymt_type = 'GAP Plus'
            
    elif row['payment_category_id'] == 3:
        if row['pymt_type_id'] == 2:
            pymt_type = 'Additional TotalRestart Pymt'
        else: 
            pymt_type = 'TotalRestart'

    pymtAmt = "{:.2f}".format(row['amount'])

    if row['pymt_method'] == 'Check':
        qbxmlQuery = '''
        <?qbxml version="14.0"?>
        <QBXML>
            <QBXMLMsgsRq onError="stopOnError">
                <CheckAddRq>
                    <CheckAdd>
                        <AccountRef>
                            <FullName>{checking}</FullName>
                        </AccountRef>
                        <PayeeEntityRef>
                            <ListID>{lender_qbid}</ListID>
                        </PayeeEntityRef>
                        <TxnDate>{date}</TxnDate>
                        <Memo>{memo}</Memo>
                        <Address>
                            <Addr1>{lender}</Addr1>
                            <Addr2>{contact}</Addr2>
                            <Addr3>{address}</Addr3>
                            <City>{city}</City>
                            <State>{state}</State>
                            <PostalCode>{zip}</PostalCode>
                        </Address>
                        <IsToBePrinted>true</IsToBePrinted>
                        <ExpenseLineAdd>
                            <AccountRef>
                                <FullName>{expense}</FullName>
                            </AccountRef>
                            <Amount>{amount}</Amount>
                            <Memo>{memo}</Memo>
                        </ExpenseLineAdd>
                    </CheckAdd>
                </CheckAddRq>
            </QBXMLMsgsRq>
        </QBXML>'''.format(checking=row['checking'], lender_qbid=row['QB_ListID'], lender=row['lender_name'], date=pay_date, 
                memo=f"{row['last']}/{row['first']} {pymt_type}", contact=row['contact'], address=row['address1'], city=row['city'], state=row['state'],
                zip=row['zip'], expense=row['expense'], amount = pymtAmt)

    elif 'ACH' in row['pymt_method']:
        qbxmlQuery = '''
        <?qbxml version="14.0"?>
        <QBXML>
            <QBXMLMsgsRq onError="stopOnError">
                <CheckAddRq>
                    <CheckAdd>
                        <AccountRef>
                            <FullName>{checking}</FullName>
                        </AccountRef>
                        <PayeeEntityRef>
                            <ListID>{lender_qbid}</ListID>
                        </PayeeEntityRef>
                        <RefNumber>ACH</RefNumber>
                        <TxnDate>{date}</TxnDate>
                        <Memo>{memo}</Memo>
                        <Address>
                            <Addr1>{lender}</Addr1>
                            <Addr2>{contact}</Addr2>
                            <Addr3>{address}</Addr3>
                            <City>{city}</City>
                            <State>{state}</State>
                            <PostalCode>{zip}</PostalCode>
                        </Address>
                        <IsToBePrinted>false</IsToBePrinted>
                        <ExpenseLineAdd>
                            <AccountRef>
                                <FullName>{expense}</FullName>
                            </AccountRef>
                            <Amount>{amount}</Amount>
                            <Memo>{memo}</Memo>
                        </ExpenseLineAdd>
                    </CheckAdd>
                </CheckAddRq>            
            </QBXMLMsgsRq>
        </QBXML>'''.format(checking=row['checking'], lender_qbid=row['QB_ListID'], lender=row['lender_name'], date=pay_date, 
                memo=f"{row['last']}/{row['first']} {pymt_type}", contact=row['contact'], address=row['address1'], city=row['city'], state=row['state'],
                zip=row['zip'], expense=row['expense'], amount = pymtAmt)
        
    # Send query and receive response
    responseString = sessionManager.ProcessRequest(ticket, qbxmlQuery)

    # output TxnID
    QBXML = ET.fromstring(responseString)
    QBXMLMsgsRs = QBXML.find('QBXMLMsgsRs')
    checkResults = QBXMLMsgsRs.iter("CheckRet")
    txnId = 0
    for checkResult in checkResults:
        txnId = checkResult.find('TxnID').text

    # Add TxnID to ready_to_be_paid table
    sql_file6 = '''UPDATE ready_to_be_paid
                   SET qb_txnid = '{TxnID}'
                   WHERE rtbp_id = {rowID};'''.format(TxnID=txnId, rowID=row['rtbp_id'])
    
    # execute and commit sql
    cursor.execute(sql_file6)
    cnx.commit()


In [ ]:
# Disconnect from Quickbooks
sessionManager.EndSession(ticket)
sessionManager.CloseConnection()

In [ ]:
# close mysql connection
cursor.close()
cnx.close()

In [ ]:
# Create and print claim letter and calculation
# Import functions to create letters [GAP (1), PLUS(2), TOTALRESTART (3)] 

In [ ]:
# Add Fraud Language fields to pymts_df


In [ ]:
# Create GAP Letters
# Collect GAP payments
gap_pymts_df = pymts_df.loc[pymts_df['payment_category_id'] == 1]
gap_pymts_df.head(20)

In [ ]:
gap_letters_df = gap_pymts_df.loc[gap_pymts_df['amount'] > 0]
gap_letters_df.head(20)

In [ ]:
# # create letters for amounts greater than $ **(FUTURE) email letters if 0
# # gap_letters_df = gap_pymts_df.loc[gap_pymts_df['amount'] > 0]

# if len(gap_letters_df.index) > 0:
    
#     sql_where = ''
#     pymt_type = 'WHERE (p.payment_type_id = 1) AND ('

#     if len(gap_letters_df.index) == 1:
#         for index, row in gap_letters_df.iterrows():
#             claimID  = row['claim_id']    
#             sql_where = f"{pymt_type}c.claim_id = {claimID}"
#     else:
#         for index, row in gap_letters_df.iterrows():
#             claimID  = row['claim_id']  
#             if sql_where == '':
#                 sql_where = f"{pymt_type}c.claim_id = {claimID}"
#             else:
#                 sql_where = sql_where + f" OR c.claim_id = {claimID}"

#     # connect to DB
#     cnx = mc.connect(user=vgc_u, password=vgc_pw,
#                     host=vgc_host,
#                     database='visualgap_claims')
#     cursor = cnx.cursor()

#     sql_query = '''
#                 SELECT c.claim_nbr, c.loss_date, l.alt_name, l.contact, l.address1, l.city, l.state,
#                     l.zip, b.first, b.last, b.acct_number, p.payment_amount, f.StateDesc, f.StateCode,
#                     CAST(f.Language AS CHAR(1000) CHARACTER SET utf8) AS f_lang
#                 FROM claims c
#                 INNER JOIN claim_lender l
#                         USING (claim_id)
#                 INNER JOIN claim_borrower b
#                         USING (claim_id)
#                 INNER JOIN claim_payments p
#                         USING (claim_id)
#                 LEFT JOIN FraudLang f
#                         ON (l.state = f.StateId)
#                 {sqlWhere});'''.format(sqlWhere = sql_where)

#     print(sql_query)

#     cursor.execute(sql_query)
#     # save query results as DF
#     template_df = pd.DataFrame(cursor.fetchall())
#     # add column names to DF
#     num_cols = len(cursor.description)
#     col_names = [i[0] for i in cursor.description]
#     template_df.columns = col_names

#     # close mysql connection
#     cursor.close()
#     cnx.close()

#     file_list = gap_letter(template_df)

#     # Concatenated output file
#     outfn = f'letters/{now.strftime("%Y-%m-%d")} GAP.pdf'

#     # Concatenate pdfs
#     ConCat_pdf(file_list, outfn)
        
# else:
#     print('No amount greater then 0.')

# # template_df.head(20)